In [26]:
import matplotlib.pyplot as plt
import numpy as np
import os, argparse, json
from argparse import Namespace
import time

import sys
sys.path.append(os.path.abspath("/home/aarushg"))

import keras_model as models
import get_dataset as str_ww_data
import str_ww_util as util

from torch.utils.data import DataLoader, Dataset
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

from efficient_kan import KAN

from tqdm import tqdm
import mplhep as hep
hep.style.use("CMS")

from torch.utils.data import TensorDataset, DataLoader
torch.cuda.empty_cache()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

from KAN_Conv_Impl.KANLinear import KANLinear
from KAN_Conv_Impl.KANConv import KANConv 

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [27]:
from types import SimpleNamespace

flags = SimpleNamespace(**dict(num_background_clips=50, l2_reg=0.001, min_snr_training=0.5, min_snr_validation=0.5, min_snr_test=0.5, fraction_target_training=0.2, fraction_target_validation=0.2, fraction_target_test=0.2, fraction_silent_training=0.1, fraction_silent_validation=0.1, fraction_silent_test=0.1, foreground_volume_min_training=0.25, foreground_volume_min_validation=0.25, foreground_volume_min_test=0.25, foreground_volume_max_training=1.5, foreground_volume_max_validation=1.5, foreground_volume_max_test=1.5, background_volume_training=2.0, background_volume_validation=1.5, background_volume_test=0.0, background_frequency=0.8, time_shift_ms=100.0, clip_duration_ms=1000, window_size_ms=64.0, window_stride_ms=32.0, dct_coefficient_count=40, num_classes=3, num_samples_training=100000, num_samples_validation=100000, num_samples_test=-1, batch_size=100, sample_rate=16000, learning_rate=0.001, use_qat=True, epochs=5, pretrain_epochs=3, run_test_set=True, saved_model_path='trained_models/str_ww_model.h5', model_init_path=None, lr_sched_name='reduce_on_plateau', plot_dir='./plots', speech_commands_path='/home/aarushg/KAN-FPGA/bench_marks/streaming_wakeword/data/speech_commands_v0.02', musan_path='/home/aarushg/KAN-FPGA/bench_marks/streaming_wakeword/data/musan'))

In [28]:
def preprocess(data):
    x, y = data[0].numpy(), data[1].numpy()
    torch_x = torch.from_numpy(x)
    # return torch_x.view(torch_x.shape[1], torch_x.shape[0], *torch_x.shape[2:]), y
    return torch_x.transpose(0, 2).transpose(1, 2), y
    # return torch_x,y

class SimpleDataset(Dataset):
    def __init__(self, data):
        self.data = data  # Store the (x, y) pairs

    def __len__(self):
        return len(self.data)  # Number of samples

    def __getitem__(self, idx):
        return self.data[idx]  # Get a single (x, y) pair

In [29]:
BATCH_SIZE = 256
SAVE_DIR = "/home/aarushg/KAN-FPGA/bench_marks/streaming_wakeword/datasets"

start_time = time.time()

if os.path.exists(f"{SAVE_DIR}/train_dataset.pth"):

    train_dataset = torch.load(f"{SAVE_DIR}/train_dataset.pth")
    val_dataset = torch.load(f"{SAVE_DIR}/val_dataset.pth")
    test_dataset = torch.load(f"{SAVE_DIR}/test_dataset.pth")

else:

    ds_train_tf, ds_test_tf, ds_val_tf = str_ww_data.get_all_datasets(flags)

    train_dataset = SimpleDataset(list(map(preprocess, ds_train_tf.unbatch())))
    test_dataset = SimpleDataset(list(map(preprocess, ds_test_tf.unbatch())))
    val_dataset = SimpleDataset(list(map(preprocess, ds_val_tf.unbatch())))

    torch.save(train_dataset, f"{SAVE_DIR}/train_dataset.pth")
    torch.save(val_dataset, f"{SAVE_DIR}/val_dataset.pth")
    torch.save(test_dataset, f"{SAVE_DIR}/test_dataset.pth")

end_time = time.time()  # Record the end time
elapsed_time = end_time - start_time

print(f"Elapsed time (loading dataset): {elapsed_time:.6f} seconds")

start_time = time.time() 

train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True, drop_last = True)
val_loader = DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle = True, drop_last = True)
test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = True, drop_last = True)

end_time = time.time()  # Record the end time
elapsed_time = end_time - start_time

print(f"Elapsed time (constructing dataloader): {elapsed_time:.6f} seconds")

/tmp/ipykernel_86732/2515949100.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_dataset = torch.load(f"{SAVE_DIR}/train_dataset.pth")
/tmp/ipykernel_86732/25159491

Elapsed time (loading dataset): 6.466406 seconds
Elapsed time (constructing dataloader): 0.119033 seconds


In [30]:
for x, y in train_loader:
    print(x.shape)
    print(y.shape)
    break

torch.Size([256, 40, 30, 1])
torch.Size([256, 3])


In [31]:
# model = KAN([N_inputs, 256, 256, 256, N_outputs], grid_size=40, spline_order=4, grid_eps=0.03, base_activation=nn.GELU, grid_range=[-8,8])

def make_kan_cnn():

  model = nn.Sequential(
    KANConv(in_channels=40, out_channels=40, kernel_size=(3, 1), stride=(1,1), padding=(0, 0)),
    KANConv(in_channels=40, out_channels=128, kernel_size=(1, 1)),
    nn.BatchNorm2d(128),
    nn.GELU(),
    nn.Dropout(p=0.5),

    KANConv(in_channels=128, out_channels=128, kernel_size=(5, 1), stride=(1,1), padding=(0, 0)),
    KANConv(in_channels=128, out_channels=128, kernel_size=(1, 1)),
    nn.BatchNorm2d(128),
    nn.GELU(),
    nn.Dropout(p=0.5),

    KANConv(in_channels=128, out_channels=128, kernel_size=(9, 1), stride=(1,1), padding=(0, 0)),
    KANConv(in_channels=128, out_channels=128, kernel_size=(1, 1)),
    nn.BatchNorm2d(128),
    nn.GELU(),
    nn.Dropout(p=0.5),

    KANConv(in_channels=128, out_channels=128, kernel_size=(15, 1), stride=(1,1), padding=(0, 0)),
    KANConv(in_channels=128, out_channels=32, kernel_size=(1, 1)),
    nn.BatchNorm2d(32),
    nn.GELU(),
    nn.Dropout(p=0.5),

    nn.Flatten(),
    KANLinear(64, 3)
  ).to(device)

  print("Num parameters for KAN CNN", sum(p.numel() for p in model.parameters()))
  return model

In [32]:
# model = KAN([N_inputs, 256, 256, 256, N_outputs], grid_size=40, spline_order=4, grid_eps=0.03, base_activation=nn.GELU, grid_range=[-8,8])

def make_mini_kan_cnn():

  model = nn.Sequential(
    KANConv(in_channels=40, out_channels=32, kernel_size=(3, 1), stride=(1,1), padding=(0, 0)),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.Dropout(p=0.5),

    KANConv(in_channels=32, out_channels=32, kernel_size=(5, 1), stride=(1,1), padding=(0, 0)),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.Dropout(p=0.5),

    # KANConv(in_channels=32, out_channels=32, kernel_size=(9, 1), stride=(1,1), padding=(0, 0)),
    # nn.BatchNorm2d(32),
    # nn.ReLU(),
    # nn.Dropout(p=0.5),

    KANConv(in_channels=32, out_channels=8, kernel_size=(15, 1), stride=(1,1), padding=(0, 0)),
    nn.BatchNorm2d(8),
    nn.ReLU(),
    nn.Dropout(p=0.5),

    nn.Flatten(),
    KANLinear(80, 3)
  ).to(device)

  print("Num parameters for MINI KAN CNN", sum(p.numel() for p in model.parameters()))
  return model

In [33]:
def make_cnn():
    model = nn.Sequential(
        # First Depthwise + Pointwise Conv Block
        nn.Conv2d(in_channels=40, out_channels=40, kernel_size=(3, 1), stride=1, padding=0),  # Depthwise
        nn.Conv2d(in_channels=40, out_channels=128, kernel_size=1),  # Pointwise
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.Dropout(0.5),  # Dropout rate assumed to be 0.5

        # Second Depthwise + Pointwise Conv Block
        nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(5, 1), stride=1, padding=0),  # Depthwise
        nn.Conv2d(in_channels=128, out_channels=128, kernel_size=1),  # Pointwise
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.Dropout(0.5),

        # Third Depthwise + Pointwise Conv Block
        nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(9, 1), stride=1, padding=0),  # Depthwise
        nn.Conv2d(in_channels=128, out_channels=128, kernel_size=1),  # Pointwise
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.Dropout(0.5),

        # Fourth Depthwise + Pointwise Conv Block
        nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(15, 1), stride=1, padding=0),  # Depthwise
        nn.Conv2d(in_channels=128, out_channels=32, kernel_size=1),  # Pointwise
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Dropout(0.5),

        # Flatten and Dense layers
        nn.Flatten(),
        nn.Linear(64, 3)  # Fully connected layer with 3 output classes
    )
    return model

In [34]:
model = make_mini_kan_cnn()
model.to(device)
# Define optimizer
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
# Define learning rate scheduler
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

Num parameters for MINI KAN CNN 130544


In [25]:
train_losses = []
val_losses = []

# Define loss
criterion = nn.CrossEntropyLoss()
for epoch in range(30):
    # Train
    model.train()
    epoch_train_loss = 0  # Initialize loss for the epoch
    total_batches = 0
    with tqdm(train_loader) as pbar:
        for i, (inputs, labels) in enumerate(pbar):
            inputs = inputs.to(device)
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, labels.to(device))
            loss.backward()
            optimizer.step()

            epoch_train_loss += loss.item()
            total_batches += 1

            accuracy = (output.argmax(dim=1) == labels.argmax(dim=1).to(device)).float().mean()
            pbar.set_postfix(loss=loss.item(), accuracy=accuracy.item(), lr=optimizer.param_groups[0]['lr'])
    
    average_train_loss = epoch_train_loss / total_batches
    train_losses.append(average_train_loss)  # Record the average training loss

    # Validation
    model.eval()
    val_loss = 0
    val_accuracy = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            output = model(inputs)
            val_loss += criterion(output, labels.to(device)).item()
            val_accuracy += (
                (output.argmax(dim=1) == labels.argmax(dim=1).to(device)).float().mean().item()
            )
    val_loss /= len(val_loader)
    val_accuracy /= len(val_loader)
    val_losses.append(val_loss)

    # Update learning rate
    scheduler.step()

    print(
        f"Epoch {epoch + 1}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy}"
    )

  2%|▏         | 7/390 [00:10<09:23,  1.47s/it, accuracy=0.262, loss=1.29, lr=0.0001]